In [19]:
import os
from pypdf import PdfReader
import pandas as pd
import re
from abc import ABC, abstractmethod

In [206]:
class PageIdentifier(ABC):
    def __init__(self, base_dir, result_dir, verbose=False):
        self.base_dir = base_dir
        self.result_dir = result_dir
        self.verbose = verbose
        self.pages = {
            "Aktiva": [],
            "Passiva": [],
            "GuV": []
        }
        
    def walk_dir(self):
        for root, _, files in os.walk(self.base_dir):
            for file in files:
                if file.endswith(".pdf"):
                    filepath = os.path.join(root, file)
                    if self.verbose:
                        print(filepath)
                    
                    try:
                        self.identify_pages(filepath)
                    except Exception as e:
                        print(f"Error reading {filepath}: {e}")

    def __generate_dataframe(self):
        df_list = []
        for item in self.pages.keys():
            df = pd.DataFrame.from_records(self.pages[item], columns=['filepath', 'page'])
            df['type'] = item
            df_list.append(df)

        df = pd.concat(df_list, ignore_index=True)
        return(df)
    
    def __generate_benchmarking_dataframe(self):
        df = self.__generate_dataframe()

        df_truth = pd.read_csv("../Python/table_page_truth.csv")
        df_truth = df_truth.assign(type=df_truth['type'].str.split('&')).explode('type')
        
        df_temp = df_truth.join(
            df.set_index(['filepath', 'type']),
            on=['filepath', 'type'],
            how='left',
            rsuffix='_identified'
        )
        df_temp['filepath'] = df_temp['filepath'].str.replace(self.base_dir, '', regex=False)
        df_temp['company'] = df_temp['filepath'].str.split('/').str[0]
        df_temp['file'] = df_temp['filepath'].str.split('/').str[1]
        df_temp = df_temp.drop(columns=['filepath'])

        return df_temp

    def save_to_csv(self):
        df = self.__generate_dataframe()
        df.to_csv(self.result_dir + "_identified_pages.csv", index=False)
        
    def get_dataframe(self):
        df = self.__generate_dataframe()
        return df

    def benchmark(self, verbose=True):
        df_temp = self.__generate_benchmarking_dataframe()

        correct_identified = df_temp[df_temp['page'] == df_temp['page_identified']]
        wrong_identified = df_temp[(df_temp['page'] != df_temp['page_identified']) & (df_temp['page_identified'].notna())]
        missing = df_temp[df_temp['page_identified'].isna()]

        if verbose:
            print(f"Correctly identified: {len(correct_identified)}")
            print(f"Wrongly identified: {len(wrong_identified)}")
            print(f"Missing: {len(missing)}")

        return {'correct': correct_identified, 'wrong': wrong_identified, 'missing': missing}
        
    
    @abstractmethod
    def identify_pages(self, filepath):
        pass

In [207]:
from pytesseract import image_to_string
from pdf2image import convert_from_path

class PageIdentifierRegex(PageIdentifier):
    def __init__(self, base_dir, result_dir, regex_patterns, verbose=False, ocr=False):
        super().__init__(base_dir, result_dir, verbose)
        self.regex_patterns = regex_patterns
        self.ocr = ocr

    def identify_pages(self, filepath):
        reader = PdfReader(filepath)
        for page_num, page in enumerate(reader.pages):
            text = page.extract_text()

            if not text.strip() and self.ocr:
                # If no text is found and OCR is enabled, perform OCR
                if self.verbose:
                    print(f"Performing OCR on {filepath} page {page_num + 1}")
                images = convert_from_path(filepath, first_page=page_num + 1, last_page=page_num + 1)
                if images:
                    text = image_to_string(images[0])

            for item in self.pages.keys():
                if all(
                    re.search(term, text, re.IGNORECASE | re.S) if isinstance(term, str) else False
                    for term in self.regex_patterns[item]
                ):
                    self.pages[item].append((filepath, page_num + 1))
                    if self.verbose:
                        print(f"Found {item} in {filepath} on page {page_num + 1}")


In [ ]:
simple_regex_patterns = {
    "Aktiva": [
        r"aktiva",
        r"((20\d{2}).*(20\d{2}))"
    ],
    "Passiva": [
        r"passiva",
        r"((20\d{2}).*(20\d{2}))"
    ],
    "GuV": [
        r"gewinn",
        r"verlust",
        r"rechnung",
        r"((20\d{2}).*(20\d{2}))"
    ]
}

page_identifier_regex_dummy = PageIdentifierRegex(
    base_dir = "../Geschaeftsberichte/degewo AG/",
    # base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/regex_1",
    regex_patterns = simple_regex_patterns,
    verbose = False,
    ocr = True
)

# page_identifier_regex_dummy.walk_dir()

Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 1
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 2
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 3
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 4
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 5
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 6
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 7
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 8
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 9
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 10
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 11
Performing OCR on .

In [184]:
df_dummy = page_identifier_regex_dummy.get_dataframe()
ben_df_dummy = benchmark(df_dummy)
ben_df_dummy['missing'][ben_df_dummy['missing']['company'] == 'degewo AG']

Correctly identified: 12
Wrongly identified: 48
Missing: 249


,page,type,page_identified,company,file
202,28,Aktiva,NaN,degewo AG,201231_degewo_Konzernlagebericht_Konzernabschl...
203,29,Passiva,NaN,degewo AG,201231_degewo_Konzernlagebericht_Konzernabschl...


In [188]:
images = convert_from_path("../Geschaeftsberichte/degewo AG/201231_degewo_Konzernlagebericht_Konzernabschluss.pdf", first_page=27 + 1, last_page=27 + 1)
if images:
    text = image_to_string(images[0])

text
re.search(r"((20\d{2}).*(20\d{2}))", text, re.IGNORECASE | re.S)

<re.Match object; span=(65, 1938), match='2020\n\nAKTIVA\n\nA. Anlagevermégen\nI. Immaterie>

In [211]:
simple_regex_patterns = {
    "Aktiva": [
        r"aktiva",
        r"((20\d{2}).*(20\d{2}))"
    ],
    "Passiva": [
        r"passiva",
        r"((20\d{2}).*(20\d{2}))"
    ],
    "GuV": [
        r"gewinn",
        r"verlust",
        r"rechnung",
        r"((20\d{2}).*(20\d{2}))"
    ]
}

page_identifier_regex_1 = PageIdentifierRegex(
    # base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/regex_1",
    regex_patterns = simple_regex_patterns,
    verbose = False
)

page_identifier_regex_1.walk_dir()

In [212]:
ben_df1 = page_identifier_regex_1.benchmark()
ben_df1['missing']

Correctly identified: 199
Wrongly identified: 716
Missing: 25


,page,type,page_identified,company,file
62,16,GuV,NaN,IBB,ibb_digitaler_geschaeftsbericht_2017.pdf
63,17,GuV,NaN,IBB,ibb_digitaler_geschaeftsbericht_2017.pdf
166,18,Aktiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2011_BBB.pdf
167,19,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2011_BBB.pdf
169,28,Aktiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2018_BBB.pdf
170,29,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2018_BBB.pdf
175,32,Aktiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2016_BBB.pdf
176,33,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2016_BBB.pdf
178,26,Aktiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2009_BBB.pdf
179,27,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2009_BBB.pdf


In [135]:
df_truth = pd.read_csv("../Python/table_page_truth.csv")
df_truth = df_truth.assign(type=df_truth['type'].str.split('&')).explode('type')


In [140]:
def benchmark(df):
    df_temp = df_truth.join(
        df.set_index(['filepath', 'type']),
        on=['filepath', 'type'],
        how='left',
        rsuffix='_identified'
    )
    df_temp['filepath'] = df_temp['filepath'].str.replace(page_identifier_regex_1.base_dir, '', regex=False)
    df_temp['company'] = df_temp['filepath'].str.split('/').str[0]
    df_temp['file'] = df_temp['filepath'].str.split('/').str[1]
    df_temp = df_temp.drop(columns=['filepath'])

    correct_identified = df_temp[df_temp['page'] == df_temp['page_identified']]
    wrong_identified = df_temp[(df_temp['page'] != df_temp['page_identified']) & (df_temp['page_identified'].notna())]
    missing = df_temp[df_temp['page_identified'].isna()]

    print(f"Correctly identified: {len(correct_identified)}")
    print(f"Wrongly identified: {len(wrong_identified)}")
    print(f"Missing: {len(missing)}")

    return {'correct': correct_identified, 'wrong': wrong_identified, 'missing': missing}

In [ ]:
df1 = page_identifier_regex_1.get_dataframe()
ben_df1 = benchmark(df1)
ben_df1['missing']

Correctly identified: 199
Wrongly identified: 716
Missing: 25


,page,type,page_identified,company,file
62,16,GuV,NaN,IBB,ibb_digitaler_geschaeftsbericht_2017.pdf
63,17,GuV,NaN,IBB,ibb_digitaler_geschaeftsbericht_2017.pdf
166,18,Aktiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2011_BBB.pdf
167,19,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2011_BBB.pdf
169,28,Aktiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2018_BBB.pdf
170,29,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2018_BBB.pdf
175,32,Aktiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2016_BBB.pdf
176,33,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2016_BBB.pdf
178,26,Aktiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2009_BBB.pdf
179,27,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2009_BBB.pdf


In [173]:
text = PdfReader('../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf').pages[25].extract_text()
text

''

In [187]:
re.search(r"((20\d{2}).*(20\d{2}))", text, re.IGNORECASE | re.S)

<re.Match object; span=(65, 1938), match='2020\n\nAKTIVA\n\nA. Anlagevermégen\nI. Immaterie>

In [171]:
regex_patterns_2 = {
    "Aktiva": [
        r"aktiva|aktivseite|a k t i v a|a k t i v s e i t e",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr"
    ],
    "Passiva": [
        r"passiva|passivseite|p a s s i v a|p a s s i v s e i t e",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr"
    ],
    "GuV": [
        r"gewinn",
        r"verlust",
        r"rechnung",
        r"((20\d{2}).*(20\d{2}))|vorjahr"
    ]
}

page_identifier_regex_2 = PageIdentifierRegex(
    # base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/regex_2",
    regex_patterns = regex_patterns_2,
    verbose = False
)

page_identifier_regex_2.walk_dir()

In [ ]:
ben_df2 = page_identifier_regex_2.benchmark()
ben_df2['missing']

Correctly identified: 235
Wrongly identified: 805
Missing: 12


,page,type,page_identified,company,file
178,26,Aktiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2009_BBB.pdf
179,27,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2009_BBB.pdf
185,17,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2007_BBB.pdf
187,24,Aktiva,NaN,degewo AG,2017_degewo_degewo_Homepage.pdf
188,25,Passiva,NaN,degewo AG,2017_degewo_degewo_Homepage.pdf
189,27,GuV,NaN,degewo AG,2017_degewo_degewo_Homepage.pdf
190,26,Aktiva,NaN,degewo AG,2018_degewo_Konzernlagebericht_und_Konzernabsc...
191,27,Passiva,NaN,degewo AG,2018_degewo_Konzernlagebericht_und_Konzernabsc...
192,29,GuV,NaN,degewo AG,2018_degewo_Konzernlagebericht_und_Konzernabsc...
193,26,Aktiva,NaN,degewo AG,2019_degewo_Konzernbericht_und_Gleicher_Lohn_f...


In [ ]:
# Aktiva und Passiva in einigen AfStatBBB nicht korrekt formatiert
regex_patterns_3 = {
    "Aktiva": [
        r"aktiva|aktivseite|a k t i v a|a k t i v s e i t e|anlageverm.{1,2}gen",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr"
    ],
    "Passiva": [
        r"passiva|passivseite|p a s s i v a|p a s s i v s e i t e|eigenkapital",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr"
    ],
    "GuV": [
        r"gewinn",
        r"verlust",
        r"rechnung",
        r"((20\d{2}).*(20\d{2}))|vorjahr"
    ]
}

page_identifier_regex_3 = PageIdentifierRegex(
    # base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/regex_3",
    regex_patterns = regex_patterns_3,
    verbose = False
)

page_identifier_regex_3.walk_dir()

In [ ]:
ben_df3 = page_identifier_regex_3.benchmark()
ben_df3['missing']

Correctly identified: 238
Wrongly identified: 1640
Missing: 9


,page,type,page_identified,company,file
187,24,Aktiva,NaN,degewo AG,2017_degewo_degewo_Homepage.pdf
188,25,Passiva,NaN,degewo AG,2017_degewo_degewo_Homepage.pdf
189,27,GuV,NaN,degewo AG,2017_degewo_degewo_Homepage.pdf
190,26,Aktiva,NaN,degewo AG,2018_degewo_Konzernlagebericht_und_Konzernabsc...
191,27,Passiva,NaN,degewo AG,2018_degewo_Konzernlagebericht_und_Konzernabsc...
192,29,GuV,NaN,degewo AG,2018_degewo_Konzernlagebericht_und_Konzernabsc...
193,26,Aktiva,NaN,degewo AG,2019_degewo_Konzernbericht_und_Gleicher_Lohn_f...
194,27,Passiva,NaN,degewo AG,2019_degewo_Konzernbericht_und_Gleicher_Lohn_f...
195,29,GuV,NaN,degewo AG,2019_degewo_Konzernbericht_und_Gleicher_Lohn_f...


Findet nun alles, was kein OCR braucht, aber dafür auch doppelt so viel irrelevantes.

In [ ]:
regex_patterns_3 = {
    "Aktiva": [
        r"aktiva|aktivseite|a k t i v a|a k t i v s e i t e|anlageverm.{1,2}gen",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr"
    ],
    "Passiva": [
        r"passiva|passivseite|p a s s i v a|p a s s i v s e i t e|eigenkapital",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr"
    ],
    "GuV": [
        r"gewinn",
        r"verlust",
        r"rechnung",
        r"((20\d{2}).*(20\d{2}))|vorjahr"
    ]
}

page_identifier_regex_3_ocr = PageIdentifierRegex(
    # base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/regex_3",
    regex_patterns = regex_patterns_3,
    verbose = True,
    ocr = True
)

# page_identifier_regex_3_ocr.walk_dir()

../Geschaeftsberichte/Berliner Bäder Betriebe/GB_BBB_Infra_2022_low.pdf
Found Passiva in ../Geschaeftsberichte/Berliner Bäder Betriebe/GB_BBB_Infra_2022_low.pdf on page 5
Found Aktiva in ../Geschaeftsberichte/Berliner Bäder Betriebe/GB_BBB_Infra_2022_low.pdf on page 10
Found Passiva in ../Geschaeftsberichte/Berliner Bäder Betriebe/GB_BBB_Infra_2022_low.pdf on page 11
Found GuV in ../Geschaeftsberichte/Berliner Bäder Betriebe/GB_BBB_Infra_2022_low.pdf on page 13
Found Aktiva in ../Geschaeftsberichte/Berliner Bäder Betriebe/GB_BBB_Infra_2022_low.pdf on page 15
Found Passiva in ../Geschaeftsberichte/Berliner Bäder Betriebe/GB_BBB_Infra_2022_low.pdf on page 15
Found GuV in ../Geschaeftsberichte/Berliner Bäder Betriebe/GB_BBB_Infra_2022_low.pdf on page 15
Found Aktiva in ../Geschaeftsberichte/Berliner Bäder Betriebe/GB_BBB_Infra_2022_low.pdf on page 22
Found Passiva in ../Geschaeftsberichte/Berliner Bäder Betriebe/GB_BBB_Infra_2022_low.pdf on page 22
Found Passiva in ../Geschaeftsberichte/B

In [ ]:
ben_df3_ocr = page_identifier_regex_3_ocr.benchmark()
ben_df3_ocr['missing']

Correctly identified: 247
Wrongly identified: 1705
Missing: 0


,page,type,page_identified,company,file


Mit OCR und breitem Regex, könnenalle Seiten identifiziert werden. Es müssen dann aber noch alle herausgefiltert werden, die fälschlicher Weise identifiziert wurden.

In [198]:
file_list = []
for root, _, files in os.walk("../Geschaeftsberichte/"):
    for file in files:
        file_list.append(os.path.join(root, file))

file_list

missing_files = [file for file in file_list if file not in df_truth['filepath'].values]
missing_files

['../Geschaeftsberichte/degewo AG/240618-degewo-konzernlagerbericht-konzernabschluss-2023.pdf']

The is no table of interest in  "../Geschaeftsberichte/degewo AG/240618-degewo-konzernlagerbericht-konzernabschluss-2023.pdf"